In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch # Importing pytorch
from torchvision import models, transforms # To get pretrained deep NNs and various image transforms
from torch.utils.data import Dataset, DataLoader # To create custom dataset classes and to batch data respectively
from torch import nn, optim # NN module to customize network and to get loss functions, optim to get various optimizers
from PIL import Image # to load images
import matplotlib.pyplot as plt # to plot images

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Custom dataset class
class DogBreedDataset(Dataset):
    def __init__(self, training_dir_path, train_df, label2id, transform):
        self.training_dir_path = training_dir_path
        self.df = train_df
        self.label2id = label2id
        self.transform = transform
        
    def __len__(self):
        # Return size of the dataset
        return len(self.df.index)
    
    def __getitem__(self, idx):
        # Read image using PIL
        image = Image.open(os.path.join(self.training_dir_path, self.df.iloc[idx, 0]) + '.jpg')
        
        # Extract image's true label from dataframe
        label = label2id[self.df.iloc[idx, 1]]
        
        # Transform image if specified (eg. Resize, Crop, flip, etc)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Get any DNN with pre-trained weights and modify classifier to output N numbers
# model = models.resnet50(pretrained=True)
# num_ftrs = model.fc.in_features
# model.fc = nn.Linear(num_ftrs, 120, bias = True)
model = models.densenet161(pretrained=True)
model.classifier = nn.Linear(2208, 120)

# for i, layer in enumerate(model.parameters()):
#     print(i, layer.shape)
#     if i < 9:
#         layer.requires_grad_ = False
# print(model)

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [ ]:
# Read csv file containing image file names and it's corresponding breed name. Also shuffle after reading
labels = pd.read_csv('/kaggle/input/dog-breed-identification/labels.csv').sample(frac=1).reset_index(drop=True)

# Get all unique breed names
unique_labels = labels.breed.unique()
unique_labels.sort()

# Make dict to map breed name to a index b.w 0 to N-1
label2id = {}
for idx, name in enumerate(unique_labels):
    label2id[name] = idx

training_dir_path = '/kaggle/input/dog-breed-identification/train'
labels_path = '/kaggle/input/dog-breed-identification/labels.csv'
    
def get_train_val_loader(labels, label2id, training_dir_path, labels_path, train_frac = 1, batch_size = 32, nb_workers = 4):
    # Total number of training examples
    nb_exmpls = len(labels.index)
    
    # Split original dataframe to train and validation df
    split = int(np.floor(train_frac * nb_exmpls))
    train_df = labels.iloc[:split, :]
    val_df = labels.iloc[split:, :]

    # Transform for train set
    train_transform = transforms.Compose([transforms.ColorJitter(brightness=0.3, contrast=0.4, saturation=0.4),
                                        transforms.Resize((224, 224)),
                                        transforms.RandomHorizontalFlip(),
                                        transforms.RandomAffine(degrees = 30, scale = (.75, 1.25)),
                                        transforms.ToTensor(),
                                        #transforms.Normalize([0, 0, 0], [1, 1, 1])])
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

    # Create dataset class for train set
    train_dataset = DogBreedDataset(training_dir_path, train_df, label2id, train_transform)
    
    # Create data loader to batch data on the fly to feed the model
    train_loader = DataLoader(train_dataset, batch_size=batch_size, num_workers=nb_workers, shuffle=True)
    
    if train_frac == 1:
        return (train_dataset, train_loader)
    else:
        # Transform for validation set
        val_transform = transforms.Compose([transforms.Resize((224, 224)),
                                            transforms.ToTensor(),
                                            #transforms.Normalize([0, 0, 0], [1, 1, 1])])
                                            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

        # Create dataset class for validation set
        val_dataset = DogBreedDataset(training_dir_path, val_df, label2id, val_transform)

        val_loader = DataLoader(val_dataset, batch_size=batch_size, num_workers=nb_workers, shuffle=True)
        
        return (train_dataset, train_loader, val_dataset, val_loader)
    
batch_size = 24
train_dataset, train_loader, val_dataset, val_loader = get_train_val_loader(labels, label2id, training_dir_path, labels_path, batch_size = batch_size, train_frac = 0.8)

In [ ]:
# Visulize images by reading a batch from loaders
# for images, targets in train_loader:
#     for i in range(batch_size):
#         img = images[i, :, :, :].numpy().transpose((1, 2, 0))
#         mean = np.array([[[.485, 0.456, 0.406]]])
#         std = np.array([[[.229, 0.224, 0.225]]])
#         img = img*std + mean
#         plt.imshow(img)
#         plt.title(unique_labels[int(targets[i].item())])
#         plt.show()
#     break

In [ ]:
# Cross entropy loss for multiclass classification and Adam optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [ ]:
# Total number of epochs to train for
n_epochs = 100

# Init max val acc to 0
val_acc_max = 0

for epoch in range(n_epochs):
    train_loss = 0.0
    val_loss = 0.0
    total = 0
    correct = 0
    
    # Set model to train mode
    model.train()
    
    # Iterate through training set by processing batches
    for data, target in train_loader:
        # Move data to GPU if available
        data, target = data.to(device), target.to(device)
        
        # Zero out gradients
        optimizer.zero_grad()
        
        # Forward pass
        output = model(data)
        
        # Make predictions and calculate correct predictions
        _, preds = torch.max(output, dim=1)
        total += target.size(0)
        correct += torch.sum(preds == target).cpu().data.item()
        
        # Calculate loss and update the weights
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)
        
    # Calculate avg loss over whole training set
    train_loss = train_loss/len(train_dataset)
    print('Epoch: {} \tTraining Loss: {:.6f} \t Training Accuracy: {:.6f}'.format(epoch+1, train_loss, correct/total))
    
    # Set model to eval mode
    model.eval()
    
    total = 0
    correct = 0
    
    # Do forward pass through validation data and calculate val. loss
    for data, target in val_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        _, preds = torch.max(output, dim=1)
        total += target.size(0)
        correct += torch.sum(preds == target).cpu().data.item()
        loss = criterion(output, target)
        val_loss += loss.item()*data.size(0)
        
    val_loss = val_loss/len(val_dataset)
    
    print('Val Loss: {:.6f} \t Val Accuracy: {:.6f}'.format(val_loss, correct/total))
    
    # save model if validation acc has increased
    if (correct/total) > val_acc_max:
        print('Validation Acc increased ({:.6f} --> {:.6f}).  Saving model ...'.format(val_acc_max, correct/total))
        torch.save(model.state_dict(), '/kaggle/working/dog_breed_model.pt')
        val_acc_max = correct/total

In [ ]:
# Get all test images' filenames into a dataframe
test_dir_path = '/kaggle/input/dog-breed-identification/test'
filenames = []
for filename in os.listdir(test_dir_path):
    filenames.append(filename.split('.')[0])
out = pd.DataFrame(filenames, columns = ['id'])
out = out.sort_values(by = 'id')
out = out.reset_index(drop=True)

In [ ]:
# Custom dataset class for test images
class TestDataset(Dataset):
    def __init__(self, test_dir_path, test_df, transform):
        self.test_dir_path = test_dir_path
        self.df = test_df
        self.transform = transform
        
    def __len__(self):
        # Return size of the dataset
        return len(self.df.index)
    
    def __getitem__(self, idx):
        # Read image using PIL
        image = Image.open(os.path.join(self.test_dir_path, self.df.iloc[idx, 0]) + '.jpg')
        
        # Transform image if specified (eg. Resize, Crop, flip, etc)
        if self.transform:
            image = self.transform(image)
        return image
    
test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                    transforms.ToTensor(),
                                    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    
batch_size = 128
test_dataset = TestDataset(test_dir_path, out, test_transform)
test_loader = DataLoader(test_dataset, batch_size=batch_size, num_workers=0)

In [ ]:
# Load saved model and run forward pass on test images
model_path = '/kaggle/working/dog_breed_model.pt'

preds = np.zeros((batch_size, len(unique_labels)), dtype=np.float32)

saved_model = models.resnet18(pretrained=False)
saved_model.fc = nn.Linear(512, 120, bias=True)
saved_model.load_state_dict(torch.load(model_path))
saved_model = saved_model.to(device)
saved_model.eval()

for data in test_loader:
    data = data.to(device)
    output = saved_model(data)
    preds = np.vstack((preds, output.detach().cpu().numpy()))
preds = preds[batch_size:, :]

In [ ]:
# Convert logits to probability distribution (softmax)
preds = np.exp(preds)
accm = np.sum(preds, axis=1)
accm = accm[:, np.newaxis]
preds /= accm

In [ ]:
# Write predictions to csv file as shown in sample submission file
preds_df = pd.DataFrame(data=preds, columns=unique_labels)
submission = pd.concat([out, preds_df], axis = 1, ignore_index=True)
submission.to_csv (r'/kaggle/working/submission.csv', index = False, header=True)